In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cpu" # the device to load the model onto

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to(device);

In [ ]:

risk_profiles = [
    "1. LOW-RISK: You are a conservative investor who is not willing to take on much risk. You are looking for investments that are safe and secure, even if they offer lower returns.",
    "2. MEDIUM-RISK: You are a moderate investor who is willing to take on some risk in exchange for the potential of higher returns. You are looking for a balanced mix of investments that offer a good combination of safety and growth.",
    "3. HIGH-RISK: You are an aggressive investor who is willing to take on a lot of risk in exchange for the potential of higher returns. You are looking for investments that offer the highest possible returns, even if they are more volatile and risky."
]
risk_profiles = '\n'.join(risk_profiles)

# user_first_response = f"I am weathly and looking to invest my money in a conservative portfolio. I am looking for investments that are safe and secure, even if they offer lower returns."
# user_first_response = f"I am poor and looking to invest my money in a high growth portfolio. I am looking for investments that are going to earn a lot of money in a short amount of time."
user_first_response = f"I am not sure how I want to invest my money. The stock market makes me nervous, but I want to make sure my money is working for me."
# user_first_response = f"Purple monkey dishwasher."
# user_risk = input("Hello! What kind of portfolio are you looking to build?")
user_risk = user_first_response + " " + f"You are a FINRA registered investment advisor. Your job is to assign me to one of three risk categories: 1. LOW-RISK, 2. MEDIUM-RISK, 3.HIGH-RISK. Given the descriptions of the three risk categories below, assign me to the risk category that best aligns with my description:\n\n### RISK CATEGORIES:\n{risk_profiles}\n\nOnly return the name of the risk category (e.g., 'LOW-RISK')."
if model_name == "mistralai/Mistral-7B-Instruct-v0.3":
    messages=[{'role': 'user', 'content': user_risk}]

In [ ]:
print(messages)

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=50, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(decoded[0].split(messages[-1]['content'])[-1].strip())

In [ ]:
print(user_first_response)
print()
for risk in ['LOW-RISK', 'MEDIUM-RISK', 'HIGH-RISK']:
    if risk.lower() in decoded[0].split(messages[-1]['content'])[-1].strip().lower():
        risk_category = risk
        break
    else:
        risk_category = "UNKNOWN"

print(f'Risk Category: {risk_category}')

In [ ]:
raise